##### Import the libraries

In [1]:
import os
import json
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from torchvision.ops import box_iou
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sklearn.metrics import precision_recall_fscore_support
import rich
import pandas as pd
from torchvision.ops import nms
from tqdm import tqdm
from ultralytics import YOLO
import cv2
import numpy as np

##### create dataset to fit the yolo requirements

In [ ]:
import os
import json
import shutil
from PIL import Image
from sklearn.model_selection import train_test_split

# Paths for input data
image_dir = "Evaluation_dataset"
annotation_path = os.path.join("Evaluation_dataset", "merged.json")  # Single JSON file

# Paths for YOLO-formatted dataset (only validation)
output_dir = "datasets"
val_images_dir = os.path.join(output_dir, "val", "images")
val_labels_dir = os.path.join(output_dir, "val", "labels")

# Create output directories
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

# Load COCO-style annotation file
with open(annotation_path) as f:
    coco_data = json.load(f)

# Create a mapping from image ID to file name and dimensions
image_info = {img["id"]: {"file_name": img["file_name"], "width": img["width"], "height": img["height"]}
              for img in coco_data["images"]}

# Organize annotations by image ID
annotations_by_image = {}
for ann in coco_data["annotations"]:
    image_id = ann["image_id"]
    if image_id not in annotations_by_image:
        annotations_by_image[image_id] = []
    
    # Convert COCO bbox to YOLO format
    x_min, y_min, bbox_width, bbox_height = ann["bbox"]
    img_width = image_info[image_id]["width"]
    img_height = image_info[image_id]["height"]

    center_x = (x_min + bbox_width / 2) / img_width
    center_y = (y_min + bbox_height / 2) / img_height
    norm_width = bbox_width / img_width
    norm_height = bbox_height / img_height

    # YOLO format: [class_id, center_x, center_y, width, height]
    yolo_annotation = f"0 {center_x} {center_y} {norm_width} {norm_height}"
    annotations_by_image[image_id].append(yolo_annotation)

# Process and save all images to the val dataset
for image_id, img_data in image_info.items():
    img_file = img_data["file_name"]
    img_path = os.path.join(image_dir, img_file)
    label_path = os.path.join(val_labels_dir, f"{os.path.splitext(img_file)[0]}.txt")

    # Copy image to val directory
    shutil.copy(img_path, val_images_dir)

    # Save YOLO annotations
    yolo_annotations = annotations_by_image.get(image_id, [])
    with open(label_path, "w") as label_file:
        label_file.write("\n".join(yolo_annotations))

print(f"Dataset organized successfully! All {len(image_info)} images are in the 'val' folder.")

#### The model performance on the evalution dataset (test dataset)

##### load yolo

In [2]:
# Load the custom-trained weights
model = YOLO('trained_models/YOLOv8_training/weights/best.pt')

##### Evaluate the model

In [ ]:
full_results = model.val(
    data="yolo_data.yaml",         # Path to data.yaml or dict specifying train/val paths
    iou = 0.5,     # Sets the Intersection Over Union (IoU) threshold for Non-Maximum Suppression (NMS).
    device = "cuda:0"
)
# print(results)

Ultralytics 8.3.58  Python-3.10.0 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 6144MiB)


val: Scanning C:\Users\mohamad\WWR\datasets\val\labels.cache... 477 images, 0 backgrounds, 0 corrupt: 100%|██████████| 477/477 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 30/30 [00:04<00:00,  6.91it/s]


                   all        477       5233      0.641      0.606      0.626       0.32
Speed: 0.1ms preprocess, 2.9ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs\detect\val25


In [ ]:
print("Mean average precision:", full_results.box.map)
print("Mean average precision at IoU=0.50:", full_results.box.map50)
print("Precision:", full_results.box.p)
print("Recall:", full_results.box.r)
print("F1 score:", full_results.box.f1)

Mean average precision: 0.3203403753663233
Mean average precision at IoU=0.50: 0.6257152877070741
Precision: [    0.64146]
Recall: [    0.60577]
F1 score: [    0.62311]


#### WWR calculation

In [7]:
# Define the path to your image
file_name = "rectified_facade_DENW11AL0000h3Gt.jpg"

image_path = os.path.join("Evaluation_subset", file_name)

In [8]:
# Load the JSON file
file_path = os.path.join("Evaluation_subset", "labels_facade_dataset_2024-06-09-08-43-50.json")
with open(file_path, 'r') as f:
    data = json.load(f)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the correct device
model = model.to(device)

##### Facade area in meters

In [10]:
# Load the Excel file
file_path = "soest_duesseldorf_combined_valid_results_with_geometry_with_width.xlsx"
df = pd.read_excel(file_path, dtype=str)  # Read as strings to avoid type issues

# Extract the gmlid from the filename
gmlid = file_name.replace("rectified_facade_", "").replace(".jpg", "")

# Search for the corresponding row
row = df[df["gmlid"] == gmlid]

if not row.empty:
    # Retrieve relevant facade height and width
    facade_height_meters = float(row["relevant_facade_height"].values[0])
    facade_width_meters = float(row["relevant_facade_width"].values[0])

    # Compute facade area
    facade_area = facade_height_meters * facade_width_meters

    rich.print(f"GMLID: {gmlid}")
    rich.print(f"Facade Height: {facade_height_meters} meters")
    rich.print(f"Facade Width: {facade_width_meters} meters")
    rich.print(f"Facade Area: {facade_area} square meters")
else:
    rich.print(f"No matching GMLID found for {gmlid}.")

GMLID: DENW11AL0000h3Gt

Facade Height: 9.081 meters

Facade Width: 11.26536719358218 meters

Facade Area: 102.30079948491978 square meters

##### image area in pixels

In [11]:
# Step 1: Find the image details for the given file name
image_details = next(item for item in data["images"] if item["file_name"] == file_name)

# Extract width and height
image_height_pixels = image_details["height"]
image_width_pixels = image_details["width"]

rich.print(f"Image Height: {image_height_pixels} pixels")
rich.print(f"Image Width: {image_width_pixels} pixels")

# Calculate the image area
image_area_pixels = image_height_pixels * image_width_pixels

# rich.print(f"Image dimensions for '{file_name}': {image_width}x{image_height}")
rich.print(f"Total image area for '{file_name}': {image_area_pixels} pixels²")

Image Height: 272 pixels

Image Width: 337 pixels

Total image area for 'rectified_facade_DENW11AL0000h3Gt.jpg': 91664 pixels²

##### Pixel size in square meters

In [12]:
# Calculate the pixel size in square meters
pixel_size_meters = (facade_width_meters / image_width_pixels) * (
        facade_height_meters / image_height_pixels)

rich.print("pixel size in meters", pixel_size_meters)

pixel size in meters 0.001116041188306421

##### Ground truth area

In [13]:
image_id = next(item["id"] for item in data["images"] if item["file_name"] == file_name)

# Step 2: Filter annotations for the image_id and category "window" (category_id = 1)
window_annotations = [
    annotation for annotation in data["annotations"]
    if annotation["image_id"] == image_id and annotation["category_id"] == 1
]

# Step 3: Calculate total area and count
windows_area_gt = sum(ann["area"] for ann in window_annotations)
num_windows = len(window_annotations)

# rich.print(f"Number of windows: {num_windows}")
rich.print(f"Total window area for '{file_name}': {windows_area_gt} pixels²")

Total window area for 'rectified_facade_DENW11AL0000h3Gt.jpg': 21784.693692249522 pixels²

##### Calculate the predicted area

In [14]:
# Run prediction
results = model.predict(source=image_path, conf=0.2)

# Extract bounding boxes
boxes = results[0].boxes.xyxy  # Get boxes in (x1, y1, x2, y2) format
scores = results[0].boxes.conf  # Get confidence scores

# Convert to a PyTorch tensor (if not already)
if not isinstance(boxes, torch.Tensor):
    boxes = torch.tensor(boxes)
if not isinstance(scores, torch.Tensor):
    scores = torch.tensor(scores)

# Set confidence threshold
threshold = 0.1
valid_boxes = boxes[scores > threshold]

# Get the count of valid boxes
num_valid_boxes = valid_boxes.shape[0]

# Calculate the area of each valid bounding box
areas = (valid_boxes[:, 2] - valid_boxes[:, 0]) * (valid_boxes[:, 3] - valid_boxes[:, 1])

# Sum the areas to get total window area
windows_area_p = areas.sum().item()

# Print the total windows area
rich.print(f"Total predicted windows area for '{image_path}': {windows_area_p} pixels²")


image 1/1 c:\Users\mohamad\WWR\Evaluation_subset\rectified_facade_DENW11AL0000h3Gt.jpg: 320x384 8 windows, 60.2ms
Speed: 1.1ms preprocess, 60.2ms inference, 2.5ms postprocess per image at shape (1, 3, 320, 384)


Total predicted windows area for 'Evaluation_subset\rectified_facade_DENW11AL0000h3Gt.jpg': 15991.98046875 pixels²

##### WWR actual and predicted in pixels

In [15]:
# Calculate ground truth WWR
wwr_gt = windows_area_gt / image_area_pixels if image_area_pixels > 0 else 0

rich.print(f"Window-to-Wall Ratio (ground truth) (WWR) for '{file_name}': {wwr_gt:.4f}")

# Calculate predicted WWR
wwr_P = windows_area_p / image_area_pixels if image_area_pixels > 0 else 0

rich.print(f"Window-to-Wall Ratio (predcited) (WWR) for '{file_name}': {wwr_P:.4f}")

# Calculate the absolute difference
wwr_difference = abs(wwr_gt - wwr_P)

# Optionally, calculate percentage difference
wwr_percentage_diff = (wwr_difference / wwr_gt * 100) if wwr_gt > 0 else 0

# Print the results
# rich.print(f"[bold]Difference between ground truth and predicted WWR:[/bold] {wwr_difference:.4f}")
# rich.print(f"[bold]Percentage difference:[/bold] {wwr_percentage_diff:.2f}%")

Window-to-Wall Ratio (ground truth) (WWR) for 'rectified_facade_DENW11AL0000h3Gt.jpg': 0.2377

Window-to-Wall Ratio (predcited) (WWR) for 'rectified_facade_DENW11AL0000h3Gt.jpg': 0.1745

##### WWR actual in meters

In [16]:
facade_meters = image_area_pixels * pixel_size_meters

rich.print(f"facade in meters {facade_meters} meters²")

facade in meters 102.30079948491976 meters²

In [17]:
window_meters_GT = windows_area_gt * pixel_size_meters

rich.print(f"windows in meters ground trught {window_meters_GT} meters²")

windows in meters ground trught 24.31261543518955 meters²

In [18]:
WWR_GT_actual = window_meters_GT / facade_meters

rich.print(f"WWR Ground trugth {WWR_GT_actual}")

WWR Ground trugth 0.23765811760614333

##### WWR predicted in meters

In [19]:
window_meters_P = windows_area_p * pixel_size_meters

rich.print(f"windows in meters predcited {window_meters_P} meters²")

windows in meters predcited 17.847708885716823 meters²

In [20]:
WWR_p_actual = window_meters_P / facade_meters

rich.print(f"WWR predicted {WWR_p_actual}")

WWR predicted 0.17446304403855384

#### Visualizing the results (yolo) on the Evalution subset

In [21]:
# Load and visualize the image with predictions
annotated_image = results[0].plot()  # Plot the results on the image

# Display the image using Matplotlibs
plt.figure(figsize=(10, 10))
plt.imshow(annotated_image)
plt.axis("off")
plt.show()

<Figure size 1000x1000 with 1 Axes>